In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pickle
import sys
sys.path.append("../../../Benchmark/")
from benchmarker import Benchmarker

In [ ]:
import os
os.chdir("../../../datatset/Mouse_brain_cor/")

Prepare data

In [3]:
files = ["left/Anndata.h5ad", "right/Anndata.h5ad"]
adatas = []
for i in files:
    adata = sc.read_h5ad(i)
    adatas.append(adata)

In [4]:
adatas

[AnnData object with n_obs × n_vars = 2898 × 32285
     obs: 'in_tissue', 'array_row', 'array_col', 'cell_type', 'spatial1', 'spatial2', 'batch'
     obsm: 'spatial',
 AnnData object with n_obs × n_vars = 2702 × 32285
     obs: 'in_tissue', 'array_row', 'array_col', 'cell_type', 'spatial1', 'spatial2', 'batch'
     obsm: 'spatial']

In [5]:
2898+2702

5600

In [47]:
# from benchmarker import split_adata, transform_coord
# import numpy as np
# spatial = [ad.obsm["spatial"] for ad in adatas]
# spatial[0] = spatial[0] * 0.45
# spatial[1] = spatial[1] * 1
# spatial = transform_coord(spatial, axis="x", align_mode="mid", margin_size=0)
# for i, ad in enumerate(adatas):
#     ad.obs_names = [j+f"_{i}" for j in ad.obs_names]
#     ad.obsm["spatial"] = spatial[i]

In [48]:
# adatas[0].obs["batch"] = "left"
# adatas[1].obs["batch"] = "right"

In [49]:
# concat = sc.concat(adatas)
# concat.obs["spatial1"] =  list(concat.obsm["spatial"][:,0])
# concat.obs["spatial2"] =  list(concat.obsm["spatial"][:,1])

In [50]:
# concat.obs_names_make_unique()
# concat.var_names_make_unique()

In [51]:
# concat.write("concat.h5ad")

In [52]:
bm = Benchmarker(R_conda_env="Rbase")

In [53]:
# bm.h5ad2rds(in_file=concat, out_file="concat.rds", verbose=True)

Run all method

In [ ]:
# bm.run(RDS_file_path="concat.rds", H5AD_file_path="concat.h5ad",
#        save_path="../../../result/Mouse_brain_cor/embed/", 
#        n_cluster=20, verbose=True, workers=4, methods=["STAMP"])

Evaluate

In [57]:
adata = sc.read_h5ad("concat.h5ad")

In [58]:
adata[adata.obs["batch"]=="right"]

View of AnnData object with n_obs × n_vars = 2702 × 32285
    obs: 'in_tissue', 'array_row', 'array_col', 'cell_type', 'spatial1', 'spatial2', 'batch'
    obsm: 'spatial'

In [59]:
adata

AnnData object with n_obs × n_vars = 5600 × 32285
    obs: 'in_tissue', 'array_row', 'array_col', 'cell_type', 'spatial1', 'spatial2', 'batch'
    obsm: 'spatial'

In [60]:
adata.obs["cell_type"] = "Unknown"

In [ ]:
res_dict = bm.read_result("../../result/Mouse_brain_cor/embed/", 
                          index=list(adata.obs_names), 
                          save="../../result/Mouse_brain_cor/embed_dict.pkl",
                          methods=bm.all_methods + ["STADIA"])

In [62]:
# with open("../../result/Mouse_brain_cor/embed_dict.pkl", "rb") as f:
#     res_dict = pickle.load(f)

In [63]:
# metrics = bm.cal_metrics(adata=adata, batch_key="batch", label_key="cell_type",
#                          res_dict=res_dict, methods="all", verbose=True, rep=1,
#                          min_max_scale=False, save="../../result/Mouse_brain_cor/metrics.pkl")

In [64]:
# with open("../../result/Mouse_brain_cor/metrics.pkl", "rb") as f:
#     metrics = pickle.load(f)

In [ ]:
bm.set_plot_params(params_dict={"figure.dpi": 300}, font_file_path="./Helvetica.ttf")

2025-09-01 19:04:56 - INFO - Custom font 'Helvetica' has been set


In [66]:
# metric = metrics[0].drop(['Isolated labels', 'NMI', 'ARI', 'Silhouette label',
#        'cLISI', 'Bio conservation', 'Total'], axis=1)

In [ ]:
save_dir = "../../..//figures/Mouse_brain_cor/"

In [68]:
# bm.plot_heatmap(metric_df=metric,  total_name="Batch correction",
#                 save=f"{save_dir}/summary_heatmap.pdf")

In [69]:
# from benchmarker import split_adata, transform_coord
# import numpy as np
# spatial = [adata.obsm["spatial"]]
# spatial = transform_coord(spatial, vertical=True, margin_size=0)
# # spatial = transform_coord(spatial, vertical=True, axis="y")

In [70]:
bg_dict = {i:"#D4B483" if i in bm.spatial_methods else "#5873a4" for i in bm.all_methods }
bg_dict["STADIA"] = "#D4B483"

In [71]:
# marker_score = bm.cal_marker_score(adata, label_dict=res_dict["Cluster"], 
#                                    marker_genes=["Cux2", "Rorb", "Fezf2", "Tbr1", "Ccn2",  "Cplx3", "Hpca",
#                                                  "C1ql2", "Mobp", "Krt18", "Penk", "Tcf7l2",
#                                                  ], 
#                                    re_run=True)

In [72]:
# bm.plot_marker_score(metric_df=marker_score, xlabel=None,
#                      save=f"{save_dir}/marker_score.pdf")

In [73]:
from benchmarker import split_adata, transform_coord
new_adata = split_adata(adata)
new_adata = sc.concat([new_adata[0], new_adata[1]])
spatial = [ad.obsm["spatial"] for ad in split_adata(new_adata)]
t = spatial[1].copy()
t[:, 0] = t[:, 0] * 0.85 + 1000
spatial[1] = t 
spatial = [np.concatenate(spatial)]
spatial = transform_coord(spatial, vertical=True, margin_size=0)
# spatial = transform_coord(spatial, margin_size=0, vertical=True)

In [74]:
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)

In [75]:
# bm.plot_legend(category_lst=list(adata.obs["batch"]), marker="o", ncol=1,
#             #    order=["Visium", "Stereo-seq", "Slide-seq V2"],
#                save=f"{save_dir}/batch_legend.pdf")

In [76]:
# bm.plot_legend(category_lst=list(range(20)), marker="o", ncol=4,
#             #    order=["Visium", "Stereo-seq", "Slide-seq V2"],
#                save=f"{save_dir}/cluster_legend.pdf")

In [77]:
# bm.plot_gene_exp(
#     adata=new_adata,
#     genes=["Cux2", "Rorb", "Fezf2", "Tbr1", "Ccn2" , "Hpca", "C1ql2", "Mobp"],
#     embed=spatial,
#     figsize=(16, 1.3), 
#     frameon=True, inner_gs_row=1, inner_gs_col=2, ncol=8,
#     xlabel=None, ylabel=None, only_show_left=True,
#     axis_width = 1.2, axis_color="lightgrey",
#     inner_col_wspace = -0.12,
#     outer_row_hspace=0.4, outer_col_wspace=0.05,
#     xlabel_pad=0.015,
#     sizes=[5, 5.5],
#     vmax="p99.5",
#     vmin="p80",
#     save=f"{save_dir}/marker_gene_plot.pdf")

In [78]:
# for key in res_dict["Cluster"].keys():
#     print(key, len(set(res_dict["Cluster"][key].flatten())))

In [ ]:
save_dir = "../../../Benchmark/figures/Mouse_brain_cor"

In [82]:
# bm.plot_spatial(spatial=spatial, label_dict=res_dict["Cluster"],
#                 figsize=(14, 10), frameon=True, inner_gs_row=1, inner_gs_col=1, size=10, ncol=5,
#                 xlabel="name", ylabel=None, only_show_left=True,
#                 axis_width = 1.2, axis_color="lightgrey",
#                 # order=["STADIA", "PRECAST", "SLAT", "STAMP", "scVI", "Scanorama",  "Harmony", "Seurat"],
#                 outer_row_hspace=0.22, outer_col_wspace=0.02, inner_common_camp=True,
#                 # inner_col_wspace = -0.12,
#                 background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#                 xlabel_pad=0.012, save_dpi=600,
#                 save=f"{save_dir}/all_spatial.png")

In [83]:
# bm.plot_umap(embed_dict=res_dict["UMAP"], batch_dict=res_dict["Batch"], annot_list=None, 
#              figsize=(14, 7), frameon=True, inner_gs_row=1, inner_gs_col=1, size=2, ncol=6,
#              xlabel="name", only_show_top=False,  ylabel=None, only_show_left=True,
#              background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#             #  order=["Unintegrated", "STADIA", "PRECAST", "SLAT", "STAMP", "scVI", "Scanorama",  "Harmony", "Seurat"],
#              axis_width=1.2, axis_color="lightgrey",
#              outer_col_wspace=0.05, outer_row_hspace=0.25, save_dpi=600,
#              ylabel_pad=0.015, xlabel_pad=0.013, merge=False, merge_margin_size=0.4,
#              save=f"{save_dir}/all_umap.png")